In [43]:
import pandas as pd


In [44]:
data = pd.read_excel('../../data/downloads/cleaning/18052025_All_Invoices_Cleaned_20250518_152331.xlsx', sheet_name='full_data')
data.head(2)

,site,site_description,supplier_no,supplier_name,invoice_id,invoice_no,date_posted,project_id,project_name,account,...,priority_multiple_commodities,all_invoice_priority_products_2008,any_invoice_priority_products_2008,freight_per_invoice,priority_product_total,non_priority_product_total,percentage_priority,percentage_non_priority,pct_priority_greater_than_70,invoice_total
0,SPN,Spectra Norcross,103277,William M. Bird,531030,656881,2024-03-01,2401132763,FAROPOINT LOBBY RENOVATION,2008,...,0.0,True,True,5.79,1343.25,0.0,100.0,0.0,True,1349.04
1,SPN,Spectra Norcross,103423,"Shaw Industries, Inc.",561080,9550085,2024-03-20,2401132815,PCI-SCU Griffin Carpet Replacement,2008,...,0.0,True,True,2.02,1749.32,0.0,100.0,0.0,True,1751.34


In [45]:
df = data

In [46]:


# Custom cleaning function
def clean_commodity(desc):
    desc = str(desc).strip().lower()
    if desc in ['vct', 'lvp', 'lvt']:
        return desc.upper()
    elif desc in ['carpet tiles', 'carpet tile']:
        return 'Carpet Tiles'
    elif desc in ['carpet roll', 'carpet rolls']:
        return 'Carpet Roll'
    return desc  # unmodified if not matched

# Apply cleaning to create a new column
df['normalized_description'] = df['new_commodity_description'].apply(clean_commodity)

# Define preferred commodities (in priority order)
preferred = ['VCT', 'LVP', 'LVT', 'Carpet Tiles', 'Carpet Roll']

# Filter to only rows with preferred commodities
df_preferred = df[df['normalized_description'].isin(preferred)].copy()

# Assign numeric priority
df_preferred['priority'] = df_preferred['normalized_description'].apply(lambda x: preferred.index(x))

# Sort by invoice_id and priority, then keep only the top preferred row per invoice
df_sorted = df_preferred.sort_values(by=["invoice_id", "priority"])
df_final = df_sorted.drop_duplicates(subset="invoice_id", keep="first").drop(columns=["priority"])

# Optionally save the result
df_final.to_csv("final_filtered_corrected_logic.csv", index=False)
